## *Using a linear regression model to predict material properties*


**Why?** Creating regression models that represent correlations between properties associated with materials enables engineers, scientists and students to find interesting trends on datasets and develop simple surrogate models.

**What?** In this tutorial, we will learn how to use the linear regression model from the [Scikit-learn](https://scikit-learn.org/stable/) Python library to develop simple models for correlations between properties across materials. Data will be obtained from [Pymatgen](http://pymatgen.org/) and [Mendeleev](https://mendeleev.readthedocs.io/en/stable/).

**How to use this?** This tutorial uses Python, some familiarity with programming would be beneficial but is not required. Run each code cell in order by clicking "Shift + Enter". Feel free to modify the code, or change queries to familiarize yourself with the workings on the code.

Suggested modifications and exercises are included in <font color=blue> blue</font>.

**Outline:**

1. Getting data
2. Processing and Organizing Data
3. Creating the Model
4. Plotting


**Get started:** Click "Shift-Enter" on the code cells to run! 


### 1. Getting Data

Using the queries from the [MSEML Query_Viz](MSEML_Query_Viz.ipynb) tutorial, we will create lists containing different properties of elements. In this first snippet of code we will import all relevant libraries, the elements that will be turned into cases and the properties that will serve as the attributes for the cases. The elements listed were chosen because they represent the three most common crystallographic structures, and also querying them for properties yields a dataset with no unknown values.

In [1]:
import numpy as np
import pymatgen as pymat
#import pymatgen.core.periodic_table as pymat
import mendeleev as mendel
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from random import shuffle
import matplotlib.pyplot as plt


fcc_elements = ["Ag", "Al", "Au", "Cu", "Ir", "Ni", "Pb", "Pd", "Pt", "Rh", "Th", "Yb"]
bcc_elements = ["Ba", "Cr", "Eu", "Fe", "Li", "Mn", "Mo", "Na", "Nb", "Ta", "V", "W" ]
hcp_elements = ["Be", "Ca", "Cd", "Co", "Dy", "Er", "Gd", "Hf", "Ho", "Lu", "Mg", "Re", 
                "Ru", "Sc", "Tb", "Ti", "Tl", "Tm", "Y", "Zn", "Zr"]
others = ["Sb", "Sm", "Bi", "Ce", "Sn", "Si"]
# Others (Solids): "Sb", "Sm", Bi" and "As" are Rhombohedral; "C" , "Ce" and "Sn" are Allotropic; 
# "Si" and "Ge" are Face-centered diamond-cubic;

elements = fcc_elements + bcc_elements + hcp_elements + others

# This function randomly arranges the elements so we can have representation for all groups both in the training and testing set
shuffle(elements) 

data_youngs_modulus = []
data_lattice_constant = []
data_melting_point = []
data_specific_heat = []
data_atomic_mass = []
data_CTE = []

for item in elements:
    data_youngs_modulus.append(pymat.Element(item).youngs_modulus)
    data_lattice_constant.append(mendel.element(item).lattice_constant)
    data_melting_point.append(mendel.element(item).melting_point)
    data_specific_heat.append(mendel.element(item).specific_heat)
    data_atomic_mass.append(pymat.Element(item).atomic_mass)
    data_CTE.append(pymat.Element(item).coefficient_of_linear_thermal_expansion)

In [2]:
# You can see how these lists look by printing them.

print(data_youngs_modulus)
#print(data_lattice_constant)
#print(data_melting_point)
#print(data_specific_heat)
#print(data_atomic_mass)
#print(data_CTE)

[61.0, 463.0, 70.0, 68.0, 74.0, 79.0, 13.0, 116.0, 16.0, 209.0, 108.0, 287.0, 47.0, 279.0, 56.0, 70.0, 18.0, 64.0, 45.0, 128.0, 411.0, 329.0, 168.0, 186.0, 4.9, 50.0, 69.0, 20.0, 65.0, 8.0, 74.0, 121.0, 55.0, 200.0, 275.0, 34.0, 50.0, 32.0, 50.0, 528.0, 130.0, 211.0, 105.0, 78.0, 24.0, 10.0, 55.0, 78.0, 198.0, 83.0, 447.0]


### 2. Processing and Organizing Data

Simple Linear Regression is one of the easiest models to implement, and can let you see how one property is related to another explanatory variable. As with all Machine Learning models, we will use a training set and a testing set.

##### SETS

Each of the lists we just created contains values for the properties of the elements. Analyzing the code, you can infer that the indexes in these arrays represent a specific element. For example:

    For element "Ag" with index 2, the properties would be located at:
   
    data_young_modulus [2]
    data_lattice_constant[2]
    data_melting_point[2]
    data_specific_heat[2]
    data_atomic_mass[2]
    data_CTE[2]


##### TRAINING AND TESTING SETS

With all this data, we will select 45 elements to become the training set and 6 elements to be the testing set.

The training group will be used to train or develop the model, the second group will be used for testing. This is done to avoid or check overfitting. While overfitting is not likely in a linear model it can happen in more complex models, like neural networks.

##### NORMALIZATION

Machine Learning models usually require data to be processed and normalized. In this linear model we will process the data to be in the format required by the [sklearn](https://scikit-learn.org/stable/) model, but we will not normalize as we are interested in visualizing trends in our raw data, instead of precise predictions.

In [3]:
# Here we will organize the data in a format sk-learn accepts
# We will develop linear models that relate Young's modulus and melting temperature, CTE vs. melting temperature 
# and lattice constant and melting temperature

# These would be the sets for Melting Point

melt_train = data_melting_point[:45] # This takes the first 45 entries to be the Training Set
melt_test = data_melting_point[-6:] # This takes the last 6 entries to be the Testing Set

# This Reshape function in the next two lines, turns each of the horizontal lists [ x, y, z] into a
# vertical NumPy array [[x]
#                       [y]
#                       [z]]
# This Step is required to work with the Sklearn Linear Model

melt_train = np.array(melt_train).reshape(-1,1) 
melt_test = np.array(melt_test).reshape(-1,1)

#Each data set will be divided in training and test data
# These would be the sets for Young's Modulus

young_train = data_youngs_modulus[:45]
young_test = data_youngs_modulus[-6:]
young_train = np.array(young_train).reshape(-1,1)
young_test = np.array(young_test).reshape(-1,1)

# These would be the sets for Lattice Constants

lattice_train = data_lattice_constant[:45]
lattice_test = data_lattice_constant[-6:]
lattice_train = np.array(lattice_train).reshape(-1,1)
lattice_test = np.array(lattice_test).reshape(-1,1)

# These would be the sets for Specific Heat

specheat_train = data_specific_heat[:45]
specheat_test = data_specific_heat[-6:]
specheat_train = np.array(specheat_train).reshape(-1,1)
specheat_test = np.array(specheat_test).reshape(-1,1)

# These would be the sets for Atomic Mass

mass_train = data_atomic_mass[:45]
mass_test = data_atomic_mass[-6:]
mass_train = np.array(mass_train).reshape(-1,1)
mass_test = np.array(mass_test).reshape(-1,1)

# These would be the sets for CTE

coefTE_train = data_CTE[:45]
coefTE_test = data_CTE[-6:]
coefTE_train = np.array(coefTE_train).reshape(-1,1)
coefTE_test = np.array(coefTE_test).reshape(-1,1)

### 3. Function to define model, train it and make predictions

Sklearn uses an "Ordinary least squares linear regression" for its Linear model. Implementation is described [here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html). We will run it with all default values. 

The **regression** function in the next cell creates a model. It does so by declaring a linear model, and then fitting it to the training set. We then make predictions feeding the model the X-axis testing set. 

We can then use the trained model (represented as an equation) to look at the variance score and understand how well our model does.

In [4]:
# This function defines a model, trains it, and uses it to predict
# It also outputs the linear model and information about its accuracy

def regression(x_train, x_test, y_train, y_test):
    
    # Define the model and train it
    model = linear_model.LinearRegression()
    model.fit(x_train, y_train)
    
    #Join train + test data 
    full_x = np.concatenate((x_train, x_test), axis=0)
    full_y = np.concatenate((y_train, y_test), axis=0)
    
    # Use the model to predict the entire set of data
    predictions = model.predict(full_x) # Make it for all values
    
    # Print model and mean squared error and variance score
    print("Linear Equation: %.4e X + (%.4e)"%(model.coef_, model.intercept_))
    print("Mean squared error: %.4e" % (mean_squared_error(full_y, predictions)))
    print('Variance score: %.4f' % r2_score(full_y, predictions))    
    
    return predictions

<font color=blue> **Exercise 1**. Research and provide for definition of the mean square error and variance score in terms of the linear model and data. You can find these definitions [here](https://scikit-learn.org/stable/modules/model_evaluation.html#mean-squared-error) and [here](https://scikit-learn.org/stable/modules/model_evaluation.html#explained-variance-score).</font>

### 4. Function for plotting data

To visualize the trends that the model fit to our data, we will plot the two properties. We will use Plotly to create this plot.

 * The first cell defines a function that organizes data and generates the plot
 * The following cells train  models and plot

In [5]:
import plotly #This is the library import
import plotly.graph_objs as go # This is the graphical object (Think "plt" in Matplotlib if you have used that before)
from plotly.offline import iplot # These lines are necessary to run Plotly in Jupyter Notebooks, but not in a dedicated environment

plotly.offline.init_notebook_mode(connected=True)

def plot(x_train, x_test, y_train, y_test, x_label, y_label, predictions):
    
    # The reshape functions in the next two lines, turns each of the
    # vertical NumPy array [[x]
    #                       [y]
    #                       [z]]
    # into python lists [ x, y, z]
    
    # This step is required to create plots with plotly like we did in the previous tutorial
    
    x_train = x_train.reshape(1,-1).tolist()[0]
    x_test = x_test.reshape(1,-1).tolist()[0]
    y_train = y_train.reshape(1,-1).tolist()[0]
    y_test = y_test.reshape(1,-1).tolist()[0]    
    predictions = predictions.reshape(1,-1).tolist()[0]
    full_x_list = x_train + x_test

    
    # Now we get back to what we know. Remember, to plot in Plotly, we need a layout and at least one trace
    
    layout0= go.Layout(hovermode= 'closest', width = 800, height=600, showlegend=True,  # Hovermode establishes the way the labels that appear when you hover are arranged # Establishing a square plot width=height
    xaxis= dict(title=go.layout.xaxis.Title(text=x_label, font=dict(size=24)), zeroline= False, gridwidth= 1, tickfont=dict(size=18)), # Axis Titles. Removing the X-axis Mark. Adding a Grid
    yaxis= dict(title=go.layout.yaxis.Title(text=y_label, font=dict(size=24)), zeroline= False, gridwidth= 1, tickfont=dict(size=18)), # Axis Titles. Removing the Y-axis Mark. Adding a Grid
    legend=dict(font=dict(size=24))) # Adding a legend
    

    training = go.Scatter(x = x_train, y = y_train, mode = 'markers', 
                          marker= dict(size= 10, color= 'green'), name= "Training Data") 
    # This trace contains the values for the data in the training set
    
    actual = go.Scatter(x = x_test, y = y_test, mode = 'markers', 
                        marker= dict(size= 10, color= 'red'), name= "Testing Data") 
    # This trace contains the values for the data in the testing set

    prediction = go.Scatter(x = full_x_list, y = predictions, mode = 'lines', 
                            line = dict(color = "blue", width = 1.5),name= "Model") 
    # This trace will be the line the model fitted the data to

    data = [training, actual, prediction]
    fig= go.Figure(data, layout=layout0)
    iplot(fig)

### 5. Train models and plot results



In [6]:
predictions = regression(melt_train, melt_test, young_train, young_test) 
# This line calls the Regression model implemented in the function 

plot(melt_train, melt_test, young_train, young_test, "Melting Temperature (K)", "Young's Modulus (GPa)", predictions) 
# This line plots the results from that model

Linear Equation: 1.1683e-01 X + (-6.9365e+01)
Mean squared error: 7.5364e+03
Variance score: 0.5223


In [7]:
predictions = regression(melt_train, melt_test, lattice_train, lattice_test)

plot(melt_train, melt_test, lattice_train, lattice_test, 
                       "Melting Temperature (K)", "Lattice Constant (Å)", predictions)

Linear Equation: -5.2411e-04 X + (4.7683e+00)
Mean squared error: 1.5359e+00
Variance score: 0.1059


In [8]:
predictions = regression(melt_train, melt_test, coefTE_train, coefTE_test)

plot(melt_train, melt_test, coefTE_train, coefTE_test, "Melting Temperature (K)", "Coefficient of Linear Thermal Expansion (K<sup>-1</sup>)", predictions)

Linear Equation: -8.4423e-09 X + (2.8145e-05)
Mean squared error: 8.1841e-11
Variance score: 0.4303


 * <font color=blue> **Exercise 2**. For the three cases above. What type of correlation do you find between properties? Explain the origin of these, approximate, correlations.
 * <font color=blue> **Exercise 3**. Plot other two properties of your choice. </font>
 * <font color=blue> **Exercise 4**. Note the mean squared error in the various models. What error would you make, in average, if you used the melting temperature to predict Young's modulus? </font>